In [1]:
import logging
import time
from typing import Optional, Tuple

import serial.tools.list_ports
from serial import Serial, SerialException


In [2]:
line_counter = 1

In [4]:
def connect(port):
    baudrate: int = 250000
    timeout: int = 1
    device: Serial = Serial(port=str(port), baudrate=baudrate, timeout=timeout)

    resp = "start"

    while resp != "":
        resp = device.readline().decode("utf-8")
        if resp != "":
            print(resp.strip())
        else:
            break
        time.sleep(0.5)
    return device


def send(device, command: str, timeout: float = 60.0) -> Tuple[bool, str]:
    """
    

    Args:
        device (_type_): _description_
        command (str): _description_
        timeout (float, optional): timeout in seconds. Defaults to 60.0.
    Returns:
        Tuple[bool, str]: _description_
    """
    command = no_line(command)
    command = cs_line(command)

    if command[-1] != "\n":
        command += "\n"

    print(f"req:  {command}")
    device.write(bytearray(command, "ascii"))

    # wait for 'ok' response

    resp = []
    for i in range(60):
        time.sleep(0.2)
        resp.append(device.readline().decode("utf-8").strip())

        if len(resp) != 0:
            print(f"\r{i} resp:\t'{resp}'")

        if resp[-1] == "ok\n":
            return True, resp

    return False, ""


def send_and_await(device, command: str):
    """
    send command to Anycubic S device, then await response

    Args:
        command (str): g-code command

    Returns:
        str: response from device
    """

    command = no_line(command)
    command = cs_line(command)

    if command[-1] != "\n":
        command += "\n"

    print(f"req:  {command}")
    device.write(bytearray(command, "ascii"))

    # wait for 'ok' response

    for i in range(60):
        time.sleep(1)
        resp = device.readline().decode("utf-8")
        if len(resp) != 0:
            print(f"\r{i} resp:\t'{resp}'")
        if resp == "ok\n":
            break

    # wait for completion of sended command

    command = no_line("M400")
    command = cs_line(command)

    if command[-1] != "\n":
        command += "\n"

    print("await for M400")
    device.write(bytearray(command, "ascii"))

    # wait for 'ok' response after sending M400

    for i in range(60):
        time.sleep(1)
        resp = device.readline().decode("utf-8")
        if len(resp) != 0:
            print(f"\r{i} resp:\t'{resp}'")
        if resp == "ok\n":
            break


In [5]:
list_ports()

List all available ports:
	 port: 'COM5', desc: 'Silicon Labs CP210x USB to UART Bridge (COM5)', hwid: 'USB VID:PID=10C4:EA60 SER=0001 LOCATION=1-6.4.2


In [6]:
device = connect("COM5")


start
echo:V1.1.4
1.1.0-RC8

echo: Last Updated: 2016-12-06 12:00 | Author: (Jolly, xxxxxxxx.CO.)
Compiled: Apr  7 2020
echo: Free Memory: 3109  PlannerBufferBytes: 1168
echo:Hardcoded Default Settings Loaded
echo:Steps per unit:
echo:  M92 X80.00 Y80.00 Z400.00 E384.00
echo:Maximum feedrates (mm/s):
echo:  M203 X500.00 Y500.00 Z8.00 E60.00
echo:Maximum Acceleration (mm/s2):
echo:  M201 X3000 Y2000 Z60 E10000
echo:Accelerations: P=printing, R=retract and T=travel
echo:  M204 P3000.00 R3000.00 T3000.00
echo:Advanced variables: S=Min feedrate (mm/s), T=Min travel feedrate (mm/s), B=minimum segment time (ms), X=maximum XY jerk (mm/s),  Z=maximum Z jerk (mm/s),  E=maximum E jerk (mm/s)
echo:  M205 S0.00 T0.00 B20000 X10.00 Y10.00 Z0.40 E5.00
echo:Home offset (mm)
echo:  M206 X0.00 Y0.00 Z0.00
echo:Z2 Endstop adjustment (mm):
echo:  M666 Z0.00
echo:Material heatup parameters:
echo:  M145 S0 H180 B70 F0
M145 S1 H240 B110 F0
echo:PID settings:
echo:  M301 P15.00 I0.95 D65.00
echo:Filament set

In [7]:
send_and_await(device, "G28")
send_and_await(device, "G1 X0 Y0 Z0")
send_and_await(device, "G1 X100 Y100 Z100")


req:  N1 G28 N1*77

12 resp:	'X:-5.00 Y:0.00 Z:0.00 E:0.00 Count X: -400 Y:0 Z:0
'
13 resp:	'ok
'
await for M400
0 resp:	'ok
'
req:  N3 G1 X0 Y0 Z0 N3*61

0 resp:	'ok
'
await for M400
0 resp:	'ok
'
req:  N5 G1 X100 Y100 Z100 N5*60

0 resp:	'ok
'
await for M400
5 resp:	'ok
'


In [8]:
device.close()